### Schema comparison within BRA

In [1]:
import pandas as pd
import chardet
from fuzzywuzzy import fuzz

In [2]:
with open('RAW Data/BRAantineoplasticos1.csv', 'rb') as rawdata:
    res = chardet.detect(rawdata.read(100000))
BRA_enc = res['encoding']

In [3]:
BRA_files=['BRAantineoplasticos1.csv', 'BRAantineoplasticos2.csv', 'BRAantineoplasticos3.csv',
          'BRACardio1.csv', 'BRACardio2.csv', 'BRACardio3.csv', 'BRAfludrocortisona.csv', 
           'BRAglibenclamida-gliclazida.csv', 'BRAinsulina.csv', 'BRAlevotiroxina.csv', 'BRAmetformina.csv']

In [4]:
BRA_files_fp = [f'RAW Data/{ds}' for ds in BRA_files]
BRA_files_fp

['RAW Data/BRAantineoplasticos1.csv',
 'RAW Data/BRAantineoplasticos2.csv',
 'RAW Data/BRAantineoplasticos3.csv',
 'RAW Data/BRACardio1.csv',
 'RAW Data/BRACardio2.csv',
 'RAW Data/BRACardio3.csv',
 'RAW Data/BRAfludrocortisona.csv',
 'RAW Data/BRAglibenclamida-gliclazida.csv',
 'RAW Data/BRAinsulina.csv',
 'RAW Data/BRAlevotiroxina.csv',
 'RAW Data/BRAmetformina.csv']

In [5]:
columns_list = []
for BRA_file in BRA_files_fp:
    data = pd.read_csv(BRA_file, header=2, encoding=BRA_enc)
    header = list(data.columns.values)
    columns_list.append(header)
assert len(columns_list) == len(BRA_files_fp)

In [6]:
BRA_fp_headers = dict(zip(BRA_files_fp, columns_list))

In [7]:
BRA_fp_headers

{'RAW Data/BRAantineoplasticos1.csv': ['Código BR',
  'Descrição\xa0CATMAT',
  '\xa0Unidade de Fornecimento\xa0',
  'Genérico',
  'Registro Anvisa',
  '\xa0Data Compra\xa0',
  'Modalidade da Compra',
  '\xa0Data Inserção',
  'Tipo Compra',
  'Fabricante',
  'CNPJ Fabricante',
  'Fornecedor',
  'CNPJ Fornecedor',
  'Nome Instituição',
  'CNPJ Instituição',
  'Município Instituição',
  'UF',
  'Qtd Itens Comprados',
  'Preço Unitário',
  'CMED - Preço Regulado',
  'Competência CMED',
  '\xa0Média Ponderada\xa0'],
 'RAW Data/BRAantineoplasticos2.csv': ['Código BR',
  'Descrição\xa0CATMAT',
  '\xa0Unidade de Fornecimento\xa0',
  'Genérico',
  'Registro Anvisa',
  '\xa0Data Compra\xa0',
  'Modalidade da Compra',
  '\xa0Data Inserção',
  'Tipo Compra',
  'Fabricante',
  'CNPJ Fabricante',
  'Fornecedor',
  'CNPJ Fornecedor',
  'Nome Instituição',
  'CNPJ Instituição',
  'Município Instituição',
  'UF',
  'Qtd Itens Comprados',
  'Preço Unitário',
  'CMED - Preço Regulado',
  'Competência CME

In [8]:
# string matching function
def fuzzy_match(list1, list2):
    res = []
    for v1 in list1:
        sim = 0
        pair = {}
        for v2 in list2:
            ratio = fuzz.ratio(v1.lower(), v2.lower())
            if ratio>sim:
                sim = ratio
                pair[v1] = v2
        print(f'pair {pair}, the score is {sim}')
        res.append(pair) 
    return res

In [9]:
Str1 = ["Apple Inc.", "Samsung", "Huawei"]
Str2 = ["apple Inc", "apple", "Sam", "huawei inc."]
res = fuzzy_match(Str1, Str2)
print(res)

pair {'Apple Inc.': 'apple Inc'}, the score is 95
pair {'Samsung': 'Sam'}, the score is 60
pair {'Huawei': 'huawei inc.'}, the score is 71
[{'Apple Inc.': 'apple Inc'}, {'Samsung': 'Sam'}, {'Huawei': 'huawei inc.'}]


In [10]:
# pairwise_compare schema 
from itertools import combinations 
def column_mapping(fp_col_dict):
    res = []
    i=0
    j=1
    items = list(fp_col_dict.items())
    for (key_1, value_1), (key_2, value_2) in combinations(fp_col_dict.items(), 2):
        fn_ij = f"{key_1.split('/')[-1].split('.')[0]}-{key_2.split('/')[-1].split('.')[0]}"
        pair_res = fuzzy_match(value_1, value_2)
        fn_pairs = {fn_ij: pair_res}
        res.append(fn_pairs)
#     for file_name_i, column_list_i in dict(list(fp_col_dict.items())[i:]).items():
#         fn_pairs = {}
#         for file_name_j, column_list_j in dict(list(fp_col_dict.items())[j:]).items():
#             fn_ij = f"{file_name_i.split('/')[-1].split('.')[0]}-{file_name_j.split('/')[-1].split('.')[0]}"
#             pair_res = fuzzy_match(column_list_i, column_list_j)
#             fn_pairs = {fn_ij: pair_res}
#             res.append(fn_pairs)
#         j+=1
#         i+=1
    return res

In [11]:
from pprint import pprint
BRA_mapping_res = column_mapping(BRA_fp_headers)


pair {'Código BR': 'Código BR'}, the score is 100
pair {'Descrição\xa0CATMAT': 'Descrição\xa0CATMAT'}, the score is 100
pair {'\xa0Unidade de Fornecimento\xa0': '\xa0Unidade de Fornecimento\xa0'}, the score is 100
pair {'Genérico': 'Genérico'}, the score is 100
pair {'Registro Anvisa': 'Registro Anvisa'}, the score is 100
pair {'\xa0Data Compra\xa0': '\xa0Data Compra\xa0'}, the score is 100
pair {'Modalidade da Compra': 'Modalidade da Compra'}, the score is 100
pair {'\xa0Data Inserção': '\xa0Data Inserção'}, the score is 100
pair {'Tipo Compra': 'Tipo Compra'}, the score is 100
pair {'Fabricante': 'Fabricante'}, the score is 100
pair {'CNPJ Fabricante': 'CNPJ Fabricante'}, the score is 100
pair {'Fornecedor': 'Fornecedor'}, the score is 100
pair {'CNPJ Fornecedor': 'CNPJ Fornecedor'}, the score is 100
pair {'Nome Instituição': 'Nome Instituição'}, the score is 100
pair {'CNPJ Instituição': 'CNPJ Instituição'}, the score is 100
pair {'Município Instituição': 'Município Instituição'}, t

In [12]:
pprint(BRA_mapping_res)

[{'BRAantineoplasticos1-BRAantineoplasticos2': [{'Código BR': 'Código BR'},
                                                {'Descrição\xa0CATMAT': 'Descrição\xa0'
                                                                        'CATMAT'},
                                                {'\xa0Unidade de Fornecimento\xa0': '\xa0'
                                                                                    'Unidade '
                                                                                    'de '
                                                                                    'Fornecimento\xa0'},
                                                {'Genérico': 'Genérico'},
                                                {'Registro Anvisa': 'Registro '
                                                                    'Anvisa'},
                                                {'\xa0Data Compra\xa0': '\xa0'
                                                            

In [13]:
print(len(BRA_mapping_res))

55


### Schema mapping within dataset CHL 

In [14]:
CHL_files = ['CHLDiab1.csv', 'CHLDiab2.csv']

In [15]:
CHL_files_fp = [f'RAW Data/{ds}' for ds in CHL_files]
CHL_files_fp

['RAW Data/CHLDiab1.csv', 'RAW Data/CHLDiab2.csv']

In [16]:
with open('RAW Data/CHLDiab1.csv', 'rb') as rawdata:
    res = chardet.detect(rawdata.read(100000))
CHL_enc = res['encoding']

In [17]:
CHL_col_list = []
for CHL_file in CHL_files_fp:
    CHL_data = pd.read_csv(CHL_file, header=0, encoding=CHL_enc)
    header = list(CHL_data.columns.values)
    CHL_col_list.append(header)
CHL_col_list

[['Unnamed: 0',
  'Codigo',
  'Tipo de Adquisicion',
  'Estado',
  'Nombre Organismo',
  'Region Unidad',
  'Codigo Moneda',
  'Fecha Creacion',
  'Fecha Publicacion',
  'Fecha Adjudicacion',
  'Fuente Financiamiento',
  'Descripcion linea Adquisicion',
  'Descripcion Proveedor',
  'Unidad Medida',
  'Nombre Proveedor',
  'Estado Oferta',
  'Cantidad Ofertada',
  'Moneda de la Oferta',
  'Monto Unitario Oferta',
  'Valor Total Ofertado',
  'Cantidad Adjudicada',
  'Monto Linea Adjudica',
  'Fecha Envio Oferta',
  'Oferta seleccionada',
  'CHL-USD',
  'Monto Unitario Oferta USD',
  'Monto Linea Adjudica USD',
  'Valor Total Ofertado USD',
  'Tipo de Institucion'],
 ['Unnamed: 0',
  'Codigo',
  'Tipo de Adquisicion',
  'Estado',
  'Nombre Organismo',
  'Region Unidad',
  'Codigo Moneda',
  'Fecha Creacion',
  'Fecha Publicacion',
  'Fecha Adjudicacion',
  'Fuente Financiamiento',
  'Descripcion linea Adquisicion',
  'Descripcion Proveedor',
  'Unidad Medida',
  'Nombre Proveedor',
  'Est

In [18]:
CHL_fp_headers = dict(zip(CHL_files_fp, CHL_col_list))

In [19]:
CHL_mapping_res = column_mapping(CHL_fp_headers)

pair {'Unnamed: 0': 'Unnamed: 0'}, the score is 100
pair {'Codigo': 'Codigo'}, the score is 100
pair {'Tipo de Adquisicion': 'Tipo de Adquisicion'}, the score is 100
pair {'Estado': 'Estado'}, the score is 100
pair {'Nombre Organismo': 'Nombre Organismo'}, the score is 100
pair {'Region Unidad': 'Region Unidad'}, the score is 100
pair {'Codigo Moneda': 'Codigo Moneda'}, the score is 100
pair {'Fecha Creacion': 'Fecha Creacion'}, the score is 100
pair {'Fecha Publicacion': 'Fecha Publicacion'}, the score is 100
pair {'Fecha Adjudicacion': 'Fecha Adjudicacion'}, the score is 100
pair {'Fuente Financiamiento': 'Fuente Financiamiento'}, the score is 100
pair {'Descripcion linea Adquisicion': 'Descripcion linea Adquisicion'}, the score is 100
pair {'Descripcion Proveedor': 'Descripcion Proveedor'}, the score is 100
pair {'Unidad Medida': 'Unidad Medida'}, the score is 100
pair {'Nombre Proveedor': 'Nombre Proveedor'}, the score is 100
pair {'Estado Oferta': 'Estado Oferta'}, the score is 10

In [20]:
pprint(CHL_mapping_res)

[{'CHLDiab1-CHLDiab2': [{'Unnamed: 0': 'Unnamed: 0'},
                        {'Codigo': 'Codigo'},
                        {'Tipo de Adquisicion': 'Tipo de Adquisicion'},
                        {'Estado': 'Estado'},
                        {'Nombre Organismo': 'Nombre Organismo'},
                        {'Region Unidad': 'Region Unidad'},
                        {'Codigo Moneda': 'Codigo Moneda'},
                        {'Fecha Creacion': 'Fecha Creacion'},
                        {'Fecha Publicacion': 'Fecha Publicacion'},
                        {'Fecha Adjudicacion': 'Fecha Adjudicacion'},
                        {'Fuente Financiamiento': 'Fuente Financiamiento'},
                        {'Descripcion linea Adquisicion': 'Descripcion linea '
                                                          'Adquisicion'},
                        {'Descripcion Proveedor': 'Descripcion Proveedor'},
                        {'Unidad Medida': 'Unidad Medida'},
                        {'Nombr

### Schema mapping within PER

In [21]:
PER_files = ['PERCardio Feb22.xls', 'PERantineoplastics1.xls', 'PERantineoplastics2.xls']

In [22]:
PER_files_fp = [f'RAW Data/{ds}' for ds in PER_files]
PER_files_fp

['RAW Data/PERCardio Feb22.xls',
 'RAW Data/PERantineoplastics1.xls',
 'RAW Data/PERantineoplastics2.xls']

In [23]:
# encoding
with open('RAW Data/PERCardio Feb22.xls', 'rb') as rawdata:
    res = chardet.detect(rawdata.read(100000))
PER_enc = res['encoding']
PER_enc

'windows-1253'

In [24]:
PER_col_list = []
for PER_file in PER_files_fp:
#     with open(PER_file, mode='r', encoding=PER_enc) as file:
    PER_data = pd.read_excel(PER_file, header=3)
    header = list(PER_data.columns.values)
    PER_col_list.append(header)
PER_col_list

[['RazonSocial',
  'Código Entidad',
  'Entidad',
  'Código Disa/Diresa',
  'Establecimiento',
  'Código Unidad Ejecutora',
  'Punto de Entrega',
  'Identificación Proceso',
  'Código Producto',
  'Producto',
  'Cantidad',
  'Precio Unitario Referencial',
  'Monto Total Referencial',
  'Precio Unitario Ofertado',
  'Monto Total Ofertado',
  'Pais'],
 ['RazonSocial',
  'Código Entidad',
  'Entidad',
  'Código Disa/Diresa',
  'Establecimiento',
  'Código Unidad Ejecutora',
  'Punto de Entrega',
  'Identificación Proceso',
  'Código Producto',
  'Producto',
  'Cantidad',
  'Precio Unitario Referencial',
  'Monto Total Referencial',
  'Precio Unitario Ofertado',
  'Monto Total Ofertado',
  'Pais'],
 ['RazonSocial',
  'Código Entidad',
  'Entidad',
  'Código Disa/Diresa',
  'Establecimiento',
  'Código Unidad Ejecutora',
  'Punto de Entrega',
  'Identificación Proceso',
  'Código Producto',
  'Producto',
  'Cantidad',
  'Precio Unitario Referencial',
  'Monto Total Referencial',
  'Precio U

In [25]:
PER_fp_headers = dict(zip(PER_files_fp, PER_col_list))

In [26]:
PER_mapping_res = column_mapping(PER_fp_headers)

pair {'RazonSocial': 'RazonSocial'}, the score is 100
pair {'Código Entidad': 'Código Entidad'}, the score is 100
pair {'Entidad': 'Entidad'}, the score is 100
pair {'Código Disa/Diresa': 'Código Disa/Diresa'}, the score is 100
pair {'Establecimiento': 'Establecimiento'}, the score is 100
pair {'Código Unidad Ejecutora': 'Código Unidad Ejecutora'}, the score is 100
pair {'Punto de Entrega': 'Punto de Entrega'}, the score is 100
pair {'Identificación Proceso': 'Identificación Proceso'}, the score is 100
pair {'Código Producto': 'Código Producto'}, the score is 100
pair {'Producto': 'Producto'}, the score is 100
pair {'Cantidad': 'Cantidad'}, the score is 100
pair {'Precio Unitario Referencial': 'Precio Unitario Referencial'}, the score is 100
pair {'Monto Total Referencial': 'Monto Total Referencial'}, the score is 100
pair {'Precio Unitario Ofertado': 'Precio Unitario Ofertado'}, the score is 100
pair {'Monto Total Ofertado': 'Monto Total Ofertado'}, the score is 100
pair {'Pais': 'Pai

In [27]:
pprint(PER_mapping_res)

[{'PERCardio Feb22-PERantineoplastics1': [{'RazonSocial': 'RazonSocial'},
                                          {'Código Entidad': 'Código Entidad'},
                                          {'Entidad': 'Entidad'},
                                          {'Código Disa/Diresa': 'Código '
                                                                 'Disa/Diresa'},
                                          {'Establecimiento': 'Establecimiento'},
                                          {'Código Unidad Ejecutora': 'Código '
                                                                      'Unidad '
                                                                      'Ejecutora'},
                                          {'Punto de Entrega': 'Punto de '
                                                               'Entrega'},
                                          {'Identificación Proceso': 'Identificación '
                                                             

### Schema mapping BRA and other files 

In [28]:
BRA_headers = dict(list(BRA_fp_headers.items())[:1])
CHL_headers = dict(list(CHL_fp_headers.items())[:1])
PER_headers = dict(list(PER_fp_headers.items())[:1])

In [29]:
PER_headers

{'RAW Data/PERCardio Feb22.xls': ['RazonSocial',
  'Código Entidad',
  'Entidad',
  'Código Disa/Diresa',
  'Establecimiento',
  'Código Unidad Ejecutora',
  'Punto de Entrega',
  'Identificación Proceso',
  'Código Producto',
  'Producto',
  'Cantidad',
  'Precio Unitario Referencial',
  'Monto Total Referencial',
  'Precio Unitario Ofertado',
  'Monto Total Ofertado',
  'Pais']}

In [30]:
DOMC_files = ['Raw Data/DOMComprasCardio.csv']

with open('Raw Data/DOMComprasCardio.csv', 'rb') as rawdata:
    res = chardet.detect(rawdata.read(100000))
DOMC_enc = res['encoding']
DOMC_enc

'utf-8'

In [31]:
DOMC_col_list = []
for DOMC_file in DOMC_files:
    DOMC_data = pd.read_csv(DOMC_file, header=0, encoding=DOMC_enc)
    header = list(data.columns.values)
    DOMC_col_list.append(header)
DOMC_fp_headers = dict(zip(DOMC_files, DOMC_col_list))

In [32]:
DOMC_headers = dict(list(DOMC_fp_headers.items())[:1])
DOMC_headers

{'Raw Data/DOMComprasCardio.csv': ['Código BR',
  'Descrição\xa0CATMAT',
  '\xa0Unidade de Fornecimento\xa0',
  'Genérico',
  'Registro Anvisa',
  '\xa0Data Compra\xa0',
  'Modalidade da Compra',
  '\xa0Data Inserção',
  'Tipo Compra',
  'Fabricante',
  'CNPJ Fabricante',
  'Fornecedor',
  'CNPJ Fornecedor',
  'Nome Instituição',
  'CNPJ Instituição',
  'Município Instituição',
  'UF',
  'Qtd Itens Comprados',
  'Preço Unitário',
  'CMED - Preço Regulado',
  'Competência CMED',
  '\xa0Média Ponderada\xa0']}

In [33]:
merge_dict = {}
merge_dict.update(BRA_headers)
merge_dict.update(CHL_headers)
merge_dict.update(PER_headers)
merge_dict.update(DOMC_headers)

In [34]:
merge_dict

{'RAW Data/BRAantineoplasticos1.csv': ['Código BR',
  'Descrição\xa0CATMAT',
  '\xa0Unidade de Fornecimento\xa0',
  'Genérico',
  'Registro Anvisa',
  '\xa0Data Compra\xa0',
  'Modalidade da Compra',
  '\xa0Data Inserção',
  'Tipo Compra',
  'Fabricante',
  'CNPJ Fabricante',
  'Fornecedor',
  'CNPJ Fornecedor',
  'Nome Instituição',
  'CNPJ Instituição',
  'Município Instituição',
  'UF',
  'Qtd Itens Comprados',
  'Preço Unitário',
  'CMED - Preço Regulado',
  'Competência CMED',
  '\xa0Média Ponderada\xa0'],
 'RAW Data/CHLDiab1.csv': ['Unnamed: 0',
  'Codigo',
  'Tipo de Adquisicion',
  'Estado',
  'Nombre Organismo',
  'Region Unidad',
  'Codigo Moneda',
  'Fecha Creacion',
  'Fecha Publicacion',
  'Fecha Adjudicacion',
  'Fuente Financiamiento',
  'Descripcion linea Adquisicion',
  'Descripcion Proveedor',
  'Unidad Medida',
  'Nombre Proveedor',
  'Estado Oferta',
  'Cantidad Ofertada',
  'Moneda de la Oferta',
  'Monto Unitario Oferta',
  'Valor Total Ofertado',
  'Cantidad Adju

In [35]:
merge_mapping_res = column_mapping(merge_dict)

pair {'Código BR': 'Codigo'}, the score is 67
pair {'Descrição\xa0CATMAT': 'Descripcion linea Adquisicion'}, the score is 40
pair {'\xa0Unidade de Fornecimento\xa0': 'Fuente Financiamiento'}, the score is 57
pair {'Genérico': 'Tipo de Institucion'}, the score is 37
pair {'Registro Anvisa': 'Region Unidad'}, the score is 64
pair {'\xa0Data Compra\xa0': 'Cantidad Ofertada'}, the score is 47
pair {'Modalidade da Compra': 'Moneda de la Oferta'}, the score is 62
pair {'\xa0Data Inserção': 'Valor Total Ofertado'}, the score is 41
pair {'Tipo Compra': 'Estado Oferta'}, the score is 50
pair {'Fabricante': 'Fecha Publicacion'}, the score is 52
pair {'CNPJ Fabricante': 'Fecha Publicacion'}, the score is 44
pair {'Fornecedor': 'Nombre Proveedor'}, the score is 54
pair {'CNPJ Fornecedor': 'Nombre Proveedor'}, the score is 52
pair {'Nome Instituição': 'Tipo de Institucion'}, the score is 69
pair {'CNPJ Instituição': 'Tipo de Institucion'}, the score is 63
pair {'Município Instituição': 'Tipo de Ins

In [36]:
pprint(merge_mapping_res)

[{'BRAantineoplasticos1-CHLDiab1': [{'Código BR': 'Codigo'},
                                    {'Descrição\xa0CATMAT': 'Descripcion linea '
                                                            'Adquisicion'},
                                    {'\xa0Unidade de Fornecimento\xa0': 'Fuente '
                                                                        'Financiamiento'},
                                    {'Genérico': 'Tipo de Institucion'},
                                    {'Registro Anvisa': 'Region Unidad'},
                                    {'\xa0Data Compra\xa0': 'Cantidad '
                                                            'Ofertada'},
                                    {'Modalidade da Compra': 'Moneda de la '
                                                             'Oferta'},
                                    {'\xa0Data Inserção': 'Valor Total '
                                                          'Ofertado'},
                        

In [46]:
import os
import csv
res_fp = 'schema-compare'
for merge_mapping_dict in merge_mapping_res:
    for files_name,value_pair in merge_mapping_dict.items():
        fname = os.path.join(res_fp, f'{files_name}.csv')
        with open(fname, 'w', newline='') as csvfile:
            files_name_sp = files_name.split('-')
            assert len(files_name_sp) == 2
            fieldnames = [files_name_sp[0], files_name_sp[1]]
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for row in value_pair:
                for v1, v2 in row.items():
                    writer.writerow({fieldnames[0]: v1, fieldnames[1]: v2})
#         file = open(fname, 'wt')
#         print(fname)
#         for item in value_pair:
#             for v1, v2 in item.items():
#                 print(f'{v1},{v2}')
#                 file.write(v1+','+v2+'\n')
#         file.close()